### Idiomatic Python: Decomposition
#### A Refactoring Exercise

Here I will mainly talk about functional decomposition, although the same principles applies to classes/methods as well.

Far too often I come across code during code reviews that contain functions (or classes), that just do way too much.

It makes the code hard to read and maintain, and, usually, very hard to unit test.

Now this is not unique to Python - it should be part of pythonic coding best practices, but applies to other languages too.

I'm sure I'll get arguments about it, but my take on writing "good" functions include:
    
1. relatively short - measured in 10's of lines, not 100's.
2. have a single purpose, and function name should clearly reflect that purpose
3. code inside a function should be at the same abstraction level - i.e. does not mix low level detail work with higher level calls to other functions. You'll see what I mean in a second.
4. should be easy to unit test (more branching code makes it more difficult to test, since you have to test not only each branch, but all the combinations of the various branches)
5. be properly documented - maybe docstrings, maybe type hints - whatever you prefer or your code base mandates as standards, but something.
6. no commented out code - that's why we have code version tools such as git - even if you work locally you can still use git.
7. no dead code (i.e. no unreachable code)
8. straightforward to understand the code in each function (i.e. no "magic" tricks and arcane incantations that hides and obfuscates how the code works)

One other thing you'll often see is that the function should not have any side effects. I guess it depends on what is meant by side effects, but I don't entirely agree with this one. 

I think the functional purists will insist on this, but Python is not for purists, and to quote the Zen of Python:

> Although practicality beats purity.

Actually in the context of writing Pythonic code, I think the entire Zen of Python bears repeating:

In [19]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


So, I think it is OK for a function to have certain side-effects - sometimes.

For example a function that acts as a logging decorator does two things - it runs and returns the value of some function call, but it also has the side effect of writing logs out somewhere. 

I consider that OK (and in fact, I don't see a way around it). On the other hand, a function that mutates one of the arguments passed to it is not being very nice - especially if it also returns a value. I would probably have strong objections to something like this (in most cases.)

I might be OK with a function that mutates an argument's state as long as it does not also return something meaningful (i.e. other than `None`). But even then, I might argue that your code design may need some re-thinking. Generally, I think a function that mutates state is probably better suited in an OOP approach with methods to mutate the instance state.

With that let's take a look at an example, and see how we can refactor it to improve it.

Here I have a function which given some arguments, queries an API for some data, transforms that data in some fashion and returns the results.

I use the Star Wars API, it is freely available and does not require us to use an API key.

We are going to need to handle paging for one of the API calls we make, as well as retries if the query times out (SWAPI being public and free with no auth required, timeouts can happen frequently, so we want to account for it).

We'll start with the bad code - a horrendous single function approach that would make anyone doing a code review stop and reject without even reading the code!!

But it works!!! you say. Sure, it works, but it is almost impossible to understand that code, and testing it is going to be really really difficult. (And to be honest, I'm not even confident my code is bug free!)

So we are going to refactor that code into something that is more readable and manageable.

Switching over to PyCharm for this. Jupyter is great for certain things, but once I get into "real" dev work, I switch to Python files and some IDE.

Note that there are many additional improvements I would make to the code - like using Pydantic to serialize the data I get back from SWAPI, maybe using a class approach to connecting to the API, etc. For now, I'll keep it simple, and if you are interested in seeing how far I would really take refactoring this, let me know in the comments. If I get enough interest and likes, I'll do a follow up video.

#### Refactor 1

First thing I'm going to do is break out the code that actually queries the API - I'm going to create a service module (and I may even create a package if I know I will have several services to contend with).

So creating a `services` package, with just the single `swapi` service, as a sub-package, and whatever modules I need under that.

Also creating a `main.py`, the standard name for the entry point of a Python app.

I'm going to keep the changes minimal, and just try to break that huge function into smaller chunks at first.

So, at the end of this refactor, the main function is already looking a whole lot simpler.

But we still have several issues.

For starters, the code in the `main()` function has different abstraction levels - I don't think the filtering of the ships belongs in that function - that should get broken out as a helper function, that way we can keep the same level of abstraction in the main function:

- get all ships (function call)
- filter ships (function call)
- return titles (function call)

Also, the code that we have in the SWAPI service for paging and query timeouts can be improved quite a bit as well. First of all the code is not DRY, and secondly, the individual functions that get the data from the API are difficult to understand because they mix things like retries and paging with the actual main logic of just getting the data. We'll need to sort that out as well.

#### Refactor 2

This refactor is just going to move the filtering logic out of the main function and into its own function.

I am going to write a specialized function that both filters the ships based on a minimum capacity, and just returns all the unique film URLs. This is a bit specialized, and one could argue that that also should be broken down into two separate functions - one to filter the ships, and one to extract just the unique film URLs. This filtering function is not really single-purpose.

So, I'm to refactor that as well.

#### Refactor 3

So with this refactor, we have the main app greatly simplified, and the filtering/extraction functions down to single purpose functions.

Note that I do not have any side effects in those functions - I send in some values, and I get something out of them. The functions do not modify my state in any way.

I could have made the `filter_ships_by_capacity` more generic, by passing in a predicate ufnction for filtering (the same way Python does it), but I'll leave that up to you if you want to do this or not.

Note how my filtering and extraction functions do not create yet more lists in memory, but rather use a generator to just yield the results. If the caller of this function needs a list, or a set, they are then free to do so, but at least we give the caller the option.

#### Refactor 4

Now it's time to turn our attention to the API service.

Our function in there do two or three things:
- paging (if required)
- timeout retry handling
- querying SWAPI

Paging we only need once, but timeout handling we do twice, so our code is not DRY. 

Although this is still much better than where we started, the whole logic of querying the SW API is totally lost inside the other "plumbing" logic of paging and query timeouts. So, I want to decompose these as well.

Let's start by stripping away the timeout handling, and focus on the paging first.

When I first started this refactor, I though I would need a decorator, but turns out I was able to get it accomplished without one - plus it's still generic enough that I could use it to page anything else in the SW API.

#### Refactor 5

Now let's look at the timeout retries.

We should be able to do this one with a decorator pretty easily.

Also, notice that I am systematically removing any handling for exceptions I can't really do anything about - later I might come back and decide if maybe I want to do some handling (like maybe if a film is not found). But for now, I'm going to let exceptions bubble the call stack and let the app exit.

#### Parting Thoughts

At this point, I don't feel entirely happy with how I implemented the paging and timeout retries - I probably could improve it, but I think it's good enough for now. I may come back to it in the future once I have time (if this were part of a project, I still would need to keep the project moving, so diminishing returns).

I would also like to add some more structure for the various data sets that are being returned and passed into the various functions. 

So, a mixture of Pydantic, maybe some named tuplesm and some type hinting.

(I don't type hint everything in my Python code - you may take a different approach - this is just me.)

Other refactoring I would consider before putting something like this in production would be taking a closer look at exception handling, and see if there are any exceptions where it might make sense to handle in some fashion.

As things are, I use `print` statements to effectively log information to stdout. Instead, this should be written using proper logging.

And of course, testing. There is no way this code would ever make it into production at any of the companies I have worked for without, at the very least, comprehensive unit tests.